In [1]:
a =[[[0, 0]]] * 3

[[[0, 0]], [[0, 0]], [[0, 0]]]

In [2]:
from mmd.common import get_start_goal_pos_circle
num_agents = 3
start_state_pos_l, goal_state_pos_l = get_start_goal_pos_circle(num_agents, 0.8)

In [3]:
start_state_pos_l

[tensor([0.8000, 0.0000], device='cuda:0'),
 tensor([-0.4000,  0.6928], device='cuda:0'),
 tensor([-0.4000, -0.6928], device='cuda:0')]

In [4]:
goal_state_pos_l

[tensor([-8.0000e-01,  9.7972e-17], device='cuda:0'),
 tensor([ 0.4000, -0.6928], device='cuda:0'),
 tensor([0.4000, 0.6928], device='cuda:0')]

In [5]:
from mmd.config.mmd_params import MMDParams as params
print(params.use_guide_on_extra_objects_only)

False


In [9]:
params.results_dir

'logs'